In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Conv2D, BatchNormalization, Dropout, Flatten
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from itertools import combinations
import timeit
import _thread
import multiprocessing
from multiprocessing import Pool

Using TensorFlow backend.


In [2]:
#turns str glich name into a 1 or 0
def gliches_get_stiches(aList,glich):
    new_list = []
    for i in range(len(aList)):
        if aList[i] == glich:
            new_list += [1]
        else:
            new_list += [0]
    return np.array(new_list)

In [3]:
def all_da_gliches(aList):
    new_list = []
    
    labels = data['label'].unique()
    for i in range(len(aList)):
        zeros = np.zeros(22)
        for x in range(len(labels)):
            if aList[i] == labels[x]:
                zeros[x] = 1

        new_list += [zeros]
    return np.array(new_list)

In [4]:
data = pd.read_csv('trainingset_v1d1_metadata.csv')
#data2 = data[data['label'] !='None_of_the_Above']
#glich to test
glich = 'Whistle'

#what features do we want 
column_selection = ["duration","peak_frequency","central_freq","amplitude","snr","bandwidth"]
#["duration","peak_frequency","central_freq","snr"]



#training data
data_train = data[data['sample_type'] == 'train' ] 
glich_list = data_train['label'].tolist()
y_train = all_da_gliches(glich_list)
x_train = np.array(data_train[column_selection])
#print(len(x_train))
#testing data
data_test = data[data['sample_type'] == 'test']
glich_list2 = data_test['label'].tolist()
y_test = all_da_gliches(glich_list2)
x_test = np.array(data_test[column_selection])

#validation data
data_val = data[data['sample_type'] == 'validation']
glich_list3 = data_val['label'].tolist()
y_val = all_da_gliches(glich_list3)
x_val = np.array(data_val[column_selection])
#print(data_test[column_selection])
x_train= np.array(list(x_train) + list(x_val))
y_train= np.array(list(y_train) + list(y_val))
print(len(x_train))
#print(data[data['label'] !='None_of_the_Above'])
#np.where(pd.isnull(data))

6787


In [25]:
#pystrain_data = pd.read_csv('ihspin_hn_0.asc')
#for i in range(len(pystrain_data)):
#    for x in pystrain_data[i]:
#        if x == "\":
#            x = ','

#np.array(pystrain_data.iloc[:,[0]])

SyntaxError: EOL while scanning string literal (<ipython-input-25-d2ca528554c2>, line 4)

In [16]:
data.head()

,event_time,ifo,peak_time,peak_time_ns,start_time,start_time_ns,duration,search,process_id,event_id,...,chisq_dof,param_one_name,param_one_value,gravityspy_id,label,sample_type,url1,url2,url3,url4
0,1.134216e+09,L1,1134216192,931639909,1134216192,832031011,0.18750,Omicron,0,21,...,0,phase,-2.72902,zmIdpucyOG,Whistle,train,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...
1,1.129360e+09,L1,1129359781,558593034,1129359781,47851085,0.94238,Omicron,0,107,...,0,phase,1.10682,zWFRqqDxwv,Whistle,test,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...
2,1.127425e+09,L1,1127425468,976317882,1127425468,960937023,0.04688,Omicron,0,218,...,0,phase,-0.83099,zKCTakFVcf,Whistle,train,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...
3,1.132637e+09,L1,1132636755,365233898,1132636754,951172113,0.82422,Omicron,0,88,...,0,phase,0.76242,z14BdoiFZS,Whistle,validation,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...
4,1.132036e+09,L1,1132035853,197264909,1132035852,933837890,2.00366,Omicron,0,16,...,0,phase,-0.31161,yyjqLCtAmO,Whistle,validation,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...,https://panoptes-uploads.zooniverse.org/produc...


In [30]:
def multi_model_trial(some_data, est):
    model3 = RandomForestClassifier(n_estimators= est)
    keep = []
    #feature_list = list(combinations(some_data,3)) + list(combinations(some_data,4)) + list(combinations(some_data,5)) + list(combinations(some_data,6))
    feature_list =  list(combinations(some_data,4)) + list(combinations(some_data,5)) + list(combinations(some_data,6))

    for i in feature_list:
        
        x_train = np.array(list(np.array(data_train[np.array(i)])) + list(np.array(data_val[np.array(i)])))
        scaler = StandardScaler().fit(x_train)
        standardized_X = scaler.transform(x_train)
        scaler2 = Normalizer().fit(standardized_X)
        std_and_norm_X_train = scaler2.transform(standardized_X)
        
        x_test = np.array(data_test[np.array(i)])
        standardized_X_test = scaler.transform(x_test)
        std_and_norm_X_test = scaler2.transform(standardized_X_test)

        #no change
        #model3.fit(x_train, y_train)
        #y_predict = model3.predict(x_test)
        
        #std 
        model3.fit(standardized_X, y_train)
        y_predict = model3.predict(standardized_X_test)
        
        #std and norm
        #model3.fit(std_and_norm_X_train, y_train)
        #y_predict = model3.predict(std_and_norm_X_test)

        keep += [y_predict]
    return keep

#get the models
the_models2 = multi_model_trial(column_selection,10)

#only keep the top models
better_model2 = []
for i in the_models2:
       if metrics.accuracy_score(y_test, i) > .8:
              better_model2.append(i)

#I did this because I'm to lazy to change vars
keep2 = []
for i in better_model2:
    keep2.append(metrics.accuracy_score(y_test, i))
print(keep2)


final2 = []
for i in range(len(better_model2[0])):
       keep2 = np.zeros(22)
       for x in range(len(better_model2)):
              keep2 += better_model2[x][i]
       for i in range(len(keep2)):
              if keep2[i] == max(keep2):
                     keep2[i] = 1
              else:
                     keep2[i] = 0
       final2.append(keep2)
final_array2 = np.array(final2)

print("Accuracy:",metrics.accuracy_score(y_test, final_array2))

[0.8015267175572519, 0.8032230703986429, 0.8023748939779474, 0.816793893129771]
Accuracy: 0.8388464800678541


In [85]:
better_model2 = []
#feature_list = list(combinations(column_selection,3)) + list(combinations(column_selection,4)) + list(combinations(column_selection,5))
feature_list =  list(combinations(column_selection,4)) + list(combinations(column_selection,5)) + list(combinations(column_selection,6)) 
num = 0
for i in the_models2:
    if metrics.accuracy_score(y_test, i) > .5:
        better_model2.append(i)
        print(round(metrics.accuracy_score(y_test, i),2),feature_list[num])
    num += 1

keep2 = []
#better_model2 = better_model2[:2]
for i in better_model2:
    keep2.append(metrics.accuracy_score(y_test, i))
print(keep2)
final2 = []
for i in range(len(better_model2[0])):
       keep2 = np.zeros(22)
       for x in range(len(better_model2)):
              keep2 += better_model2[x][i]
       for i in range(len(keep2)):
              if keep2[i] == max(keep2):
                     keep2[i] = 1
              else:
                     keep2[i] = 0
       final2.append(keep2)
final_array2 = np.array(final2)

print("Accuracy:",metrics.accuracy_score(y_test, final_array2))

0.62 ('duration', 'peak_frequency', 'central_freq', 'amplitude')
0.63 ('duration', 'peak_frequency', 'central_freq', 'snr')
0.7 ('duration', 'peak_frequency', 'central_freq', 'bandwidth')
0.71 ('duration', 'peak_frequency', 'amplitude', 'snr')
0.71 ('duration', 'peak_frequency', 'amplitude', 'bandwidth')
[0.6181506849315068, 0.6267123287671232, 0.702054794520548, 0.7123287671232876, 0.7097602739726028]
Accuracy: 0.7174657534246576


In [86]:
def multi_model_trial2(some_data):
    model3 = RandomForestClassifier(n_estimators= 100)
    keep = []
    feature_list= [['duration', 'peak_frequency', 'central_freq', 'snr'],
                    ['duration', 'peak_frequency', 'snr', 'bandwidth'],
                    ['duration', 'peak_frequency', 'central_freq', 'amplitude', 'snr'],
                    ['duration', 'peak_frequency', 'amplitude', 'snr', 'bandwidth'],
                    ['duration', 'peak_frequency', 'central_freq', 'amplitude', 'snr', 'bandwidth']]
    #feature_list = list(combinations(some_data,3)) + list(combinations(some_data,4)) + list(combinations(some_data,5)) + list(combinations(some_data,6))
    #feature_list =  list(combinations(some_data,4)) + list(combinations(some_data,5)) + list(combinations(some_data,6))

    for i in feature_list:
        
        x_train = np.array(list(np.array(data_train[np.array(i)])) + list(np.array(data_val[np.array(i)])))
        scaler = StandardScaler().fit(x_train)
        standardized_X = scaler.transform(x_train)
        scaler2 = Normalizer().fit(standardized_X)
        std_and_norm_X_train = scaler2.transform(standardized_X)
        
        x_test = np.array(data_test[np.array(i)])
        standardized_X_test = scaler.transform(x_test)
        std_and_norm_X_test = scaler2.transform(standardized_X_test)

        #no change
        #model3.fit(x_train, y_train)
        #y_predict = model3.predict(x_test)
        
        #std 
        model3.fit(standardized_X, y_train)
        #y_predict = model3.predict(standardized_X_test)
        
        #std and norm
        #model3.fit(std_and_norm_X_train, y_train)
        #y_predict = model3.predict(std_and_norm_X_test)

        keep += [y_predict]
    return keep
the_models2 = multi_model_trial2(column_selection)
better_model2 = []
for i in the_models2:
       if metrics.accuracy_score(y_test, i) > .8:
              better_model2.append(i)
keep2 = []
for i in better_model2:
    keep2.append(metrics.accuracy_score(y_test, i))
print(keep2)
final2 = []
for i in range(len(better_model2[0])):
       keep2 = np.zeros(22)
       for x in range(len(better_model2)):
              keep2 += better_model2[x][i]
       for i in range(len(keep2)):
              if keep2[i] == max(keep2):
                     keep2[i] = 1
              else:
                     keep2[i] = 0
       final2.append(keep2)
final_array2 = np.array(final2)

print("Accuracy:",metrics.accuracy_score(y_test, final_array2))

[0.836472602739726, 0.8398972602739726, 0.8407534246575342, 0.8458904109589042, 0.8373287671232876]
Accuracy: 0.8613013698630136


In [40]:
def multi_model_trial_all_in_one(some_data, est, num):
    model3 = RandomForestClassifier(n_estimators= est)
    keep = []
    the_models2 = []
    feature_list= [['duration', 'peak_frequency', 'central_freq', 'snr'],
                    ['duration', 'peak_frequency', 'snr', 'bandwidth'],
                    ['duration', 'peak_frequency', 'central_freq', 'amplitude', 'snr'],
                    ['duration', 'peak_frequency', 'amplitude', 'snr', 'bandwidth'],
                    ['duration', 'peak_frequency', 'central_freq', 'amplitude', 'snr', 'bandwidth']]
    #feature_list = list(combinations(some_data,3)) + list(combinations(some_data,4)) + list(combinations(some_data,5)) + list(combinations(some_data,6))
    #feature_list =  list(combinations(some_data,4)) + list(combinations(some_data,5)) + list(combinations(some_data,6))

    for i in feature_list:
        
        x_train = np.array(list(np.array(data_train[np.array(i)])) + list(np.array(data_val[np.array(i)])))
        scaler = StandardScaler().fit(x_train)
        standardized_X = scaler.transform(x_train)
        scaler2 = Normalizer().fit(standardized_X)
        std_and_norm_X_train = scaler2.transform(standardized_X)
        
        x_test = np.array(data_test[np.array(i)])
        standardized_X_test = scaler.transform(x_test)
        std_and_norm_X_test = scaler2.transform(standardized_X_test)

        #no change
        #model3.fit(x_train, y_train)
        #y_predict = model3.predict(x_test)
        
        #std 
        model3.fit(standardized_X, y_train)
        y_predict = model3.predict(standardized_X_test)
        
        #std and norm
        #model3.fit(std_and_norm_X_train, y_train)
        #y_predict = model3.predict(std_and_norm_X_test)
    #put all the models predictions into a list
    #TODO need to store models for later
        the_models2 += [y_predict]

    #store the models scores in a list
    model_score = []
    for i in the_models2:
        model_score.append(metrics.accuracy_score(y_test, i))  

    #pick out the models with the highest scores
    better_model3 = []
    for i in range(len(model_score)):
        if model_score[i] > num * max(model_score):
            better_model3.append(the_models2[i])

   #put the models back together by having them vote on each prediction     
    final2 = []
    for i in range(len(better_model3[0])):
        keep2 = np.zeros(22)
        for x in range(len(better_model3)):
            keep2 += better_model3[x][i]
        for i in range(len(keep2)):
            if keep2[i] == max(keep2):
                keep2[i] = 1
            else:
                keep2[i] = 0
        final2.append(keep2)
    final_array2 = np.array(final2)

    return metrics.accuracy_score(y_test, final_array2)



In [62]:
the_models2 = multi_model_trial_all_in_one(column_selection, 12, .5)
print(the_models2)

0.8490245971162002


In [43]:
multi_model_list = []
for i in range(1,51):
    multi_model_list.append(multi_model_trial_all_in_one(column_selection, i, .5))
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [44]:
print(multi_model_list)

[0.6598812553011026, 0.8006785411365565, 0.7599660729431722, 0.8269720101781171, 0.804919423240034, 0.8396946564885496, 0.8057675996607294, 0.8371501272264631, 0.825275657336726, 0.8439355385920272, 0.8379983036471587, 0.8430873621713316, 0.8396946564885496, 0.8498727735368957, 0.8422391857506362, 0.8532654792196777, 0.8456318914334181, 0.8558100084817643, 0.8524173027989822, 0.8498727735368957, 0.8515691263782866, 0.8473282442748091, 0.8498727735368957, 0.8549618320610687, 0.8524173027989822, 0.8541136556403732, 0.8464800678541137, 0.8464800678541137, 0.8541136556403732, 0.8507209499575912, 0.8490245971162002, 0.8558100084817643, 0.8558100084817643, 0.8566581849024597, 0.8490245971162002, 0.8583545377438507, 0.8524173027989822, 0.8549618320610687, 0.8507209499575912, 0.8541136556403732, 0.8541136556403732, 0.8507209499575912, 0.8566581849024597, 0.8549618320610687, 0.8532654792196777, 0.8524173027989822, 0.8558100084817643, 0.8515691263782866, 0.8549618320610687, 0.8583545377438507]


In [10]:
def single_model(some_data, est):
    model3 = RandomForestClassifier(n_estimators= est)
    feature_list = ['duration', 'peak_frequency', 'central_freq', 'amplitude', 'snr', 'bandwidth']
    scaler = StandardScaler().fit(x_train)
    standardized_X = scaler.transform(x_train)
    standardized_X_test = scaler.transform(x_test)

    model3.fit(standardized_X, y_train)
    y_predict = model3.predict(standardized_X_test)
    return metrics.accuracy_score(y_test,y_predict)

In [46]:
single_model_list2 = []
#single_model_list.append([1,single_model(column_selection, 1)])
for i in range(1,129):
    single_model_list2.append(single_model(column_selection, 2*i))
multi_model_list = []
for i in range(1,129):
    multi_model_list.append(multi_model_trial_all_in_one(column_selection, 2*i, .5))
    

In [31]:
print(single_model_list)

[[1, 0.7540288379983037], 0.6581849024597116, 0.7506361323155216, 0.7862595419847328, 0.8040712468193384, 0.7921967769296013, 0.8142493638676844, 0.8083121289228159, 0.813401187446989, 0.8193384223918575, 0.820186598812553, 0.8286683630195081, 0.821882951653944, 0.8303647158608991, 0.8227311280746396, 0.825275657336726, 0.81509754028838, 0.821882951653944, 0.820186598812553, 0.8269720101781171, 0.821882951653944, 0.825275657336726, 0.823579304495335, 0.8244274809160306, 0.8244274809160306, 0.8278201865988125, 0.8286683630195081, 0.8312128922815946, 0.8295165394402035, 0.833757421543681, 0.8329092451229856, 0.8286683630195081, 0.8244274809160306, 0.8278201865988125, 0.8295165394402035, 0.825275657336726, 0.825275657336726, 0.8286683630195081, 0.8286683630195081, 0.8269720101781171, 0.8286683630195081, 0.8329092451229856, 0.8303647158608991, 0.8286683630195081, 0.8329092451229856, 0.8388464800678541, 0.8354537743850721, 0.825275657336726, 0.825275657336726, 0.8329092451229856]


In [38]:
print(single_model_list2)

[0.7803223070398643, 0.6980491942324003, 0.7981340118744699, 0.741306191687871, 0.823579304495335, 0.7777777777777778, 0.820186598812553, 0.7828668363019508, 0.8286683630195081, 0.8057675996607294, 0.821882951653944, 0.8006785411365565, 0.8227311280746396, 0.8083121289228159, 0.825275657336726, 0.820186598812553, 0.8320610687022901, 0.8142493638676844, 0.8244274809160306, 0.8193384223918575, 0.8244274809160306, 0.8100084817642069, 0.8371501272264631, 0.8210347752332485, 0.8295165394402035, 0.816793893129771, 0.8312128922815946, 0.8227311280746396, 0.8286683630195081, 0.8278201865988125, 0.8312128922815946, 0.81509754028838, 0.8312128922815946, 0.823579304495335, 0.8278201865988125, 0.825275657336726, 0.8278201865988125, 0.8295165394402035, 0.8346055979643766, 0.821882951653944, 0.8320610687022901, 0.8210347752332485, 0.8354537743850721, 0.821882951653944, 0.8379983036471587, 0.8269720101781171, 0.8312128922815946, 0.823579304495335, 0.8346055979643766, 0.8286683630195081]


In [47]:
multi_model_list = np.array(multi_model_list)
df = pd.DataFrame(multi_model_list, columns=["score"])
df.to_csv('multi_model2.csv')

In [48]:
single_model_list = np.array(single_model_list)
df = pd.DataFrame(single_model_list, columns=["score"])
df.to_csv('single_model2.csv')

In [49]:
print(multi_model_list)
print(single_model_list)

[0.81085666 0.82866836 0.83375742 0.84308736 0.8379983  0.84393554
 0.84817642 0.84648007 0.85072095 0.84393554 0.8490246  0.8524173
 0.85496183 0.85156913 0.84648007 0.84987277 0.85665818 0.85326548
 0.85835454 0.85326548 0.85326548 0.85496183 0.85835454 0.85496183
 0.86005089 0.85072095 0.85665818 0.85581001 0.8634436  0.85665818
 0.85750636 0.8524173  0.85496183 0.85326548 0.85581001 0.85665818
 0.85750636 0.85750636 0.85835454 0.85411366 0.85496183 0.85665818
 0.85750636 0.85411366 0.85835454 0.85156913 0.86259542 0.85326548
 0.85581001 0.85665818 0.86005089 0.85496183 0.85581001 0.85581001
 0.85750636 0.85835454 0.85920271 0.85072095 0.85496183 0.85750636
 0.85156913 0.8524173  0.85156913 0.85835454 0.85920271 0.85411366
 0.85581001 0.85326548 0.86005089 0.8524173  0.85496183 0.85920271
 0.85496183 0.85156913 0.85072095 0.85750636 0.85496183 0.85665818
 0.85326548 0.8524173  0.85496183 0.85835454 0.86089907 0.85750636
 0.84987277 0.85581001 0.85920271 0.85750636 0.85496183 0.85920